### API >> https://facebook-sdk.readthedocs.io/en/latest/api.html
### Get access token >> https://developers.facebook.com/tools/explorer

In [ ]:
!pip install facebook-sdk

In [1]:
#START
#this is my token (Pitágoras)
token = "EAACEdEose0cBAPjctkkb2wngNx0TavRZCKAKTXwmtESkCmlBHMxA3QpMwwcUzYPn1n04PHxp2Bs421GM93RnNYwQXyNbkcc54ZAxtYLzipizPJsjWWrFjn9vZBYHADOgTEZBnhYpV5dYkbdPftGpqdXSDtgTt2Ark8GuO4D7oERcmYfOZB4TDCMCYV15v76wZD"
 
#ID of "Pitagoras"
otherPersonID = "10208935375967359"
raiID = "739425496128479"
anaID = "765831436837419"

In [2]:
from collections import defaultdict as ddict
import facebook as fb
import requests as req
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
#Gets the categories of pages liked by the user
#   token       user access token
#   return      dictionary of categories found, each with it's own array of pages
#       {c1 : [page1, page2, page3...], c2 : [page4, page5], c3 : [page6]...}
def getCats(token):
    cats = ddict(str)
    graph = fb.GraphAPI(access_token=token, version='2.7')
    resource = graph.get_object("me/likes?fields=name,category")
    #print(resource)
    cats = dict()
    while(True):
        for page in resource['data']:
            cat = page['category']
            if not(cat in cats):
                cats[cat] = []
            cats[cat].append(page['id'])
        # Attempt to make a request to the next page of data, if it exists.
        try:
            resource=req.get(resource['paging']['next']).json()
        except KeyError:
            #print("Finished getting pages")
            # When there are no more pages (['paging']['next']), break from the
            # loop and end the script.
            break
    return cats

In [13]:
#Get the pages that a given user liked
#   user    id of a user
#   token   access token
#   return  array of page IDs
#       [page1, page2, page3...]
def getPagesList(user, token):
    pages = list()
    graph = fb.GraphAPI(access_token=token, version='2.7')
    resource = graph.get_object(user+"/likes?fields=name")
    #print(resource)
    cats = dict()
    while(True):
        for page in resource['data']:
            pages.append(page['id'])
        # Attempt to make a request to the next page of data, if it exists.
        try:
            resource=req.get(resource['paging']['next']).json()
        except KeyError:
            #print("Finished getting pages from user " + user)
            # When there are no more pages (['paging']['next']), break from the
            # loop and end the script.
            break
    return pages


def getPagesListFriends(user, token):
    pages = list()
    try:
        graph = fb.GraphAPI(access_token=token, version='2.7')
        resource = graph.get_object(user+"?fields=likes{category,name}")["likes"]
    except:
        return pages
    cats = dict()
    while(True):
        for page in resource['data']:
            pages.append(page['id'])
        # Attempt to make a request to the next page of data, if it exists.
        try:
            resource=req.get(resource['paging']['next']).json()
        except KeyError:
            #print("Finished getting pages from user " + user)
            # When there are no more pages (['paging']['next']), break from the
            # loop and end the script.
            break
    return pages

In [14]:
#Classifies pages by categories, the category with more pages gets 1.0
#and the smaller categories get smaller and smaller classifications...
#   cats        dictionary of categories found, each with it's own array of pages
#       {c1 : [page1, page2, page3...], c2 : [page4, page5], c3 : [page6]...}
#   return      dictionary of pages, each with it's own classification (0.0 < classification <= 1.0)
#       {page1 : x, page2 : x, page4 : y, page6 : z}
def classify(cats):
    classification = ddict()
    biggestCat = ""
    biggestCatLen = 0
    for cat, pages in cats.items():
        if(len(pages) > biggestCatLen):
            biggestCat = cat
            biggestCatLen = len(pages)
    for cat, pages in cats.items():
        classify = len(pages)/biggestCatLen
        for i in pages:
            classification[i] = classify

    return classification

In [15]:
import operator

def norm(c):
    tuples = list()
    sortedByVal = sorted(c.items(), key=operator.itemgetter(1))
    
    w = [25, 18, 15, 12, 10, 8, 6, 4, 2, 1]
    for i in range(0, len(sortedByVal)-11):
        tup = sortedByVal[i]
        tuples.append((tup[0], 1))
    count = 9
    for i in range(len(sortedByVal)-9, len(sortedByVal)):
        a = w[count]
        if(i > 0):
            tup = sortedByVal[i]
            tuples.append((tup[0], a))
            count = count - 1
    newDict = dict()
    for t in tuples:
        newDict[t[0]] = t[1]
    return newDict

#d = {"11" : 0.2, "1" : 0.3, "04" : 0.4, "07" : 0.8}
#print(d)
#d1 = norm(d)
#print(d1)

In [16]:
def similarityWith(user, classification, token):
    otherUserPages = getPagesListFriends(user, token)
    rate = 0.0
    for page in otherUserPages:
        if page in classification:
            rate += classification[page]
    return rate

In [17]:
# return dict { id1 : friend_name1, id2 : friend_name2 }
def getFriends(user, token):
    graph = fb.GraphAPI(access_token=token, version='2.7')
    resource = graph.get_object("/" + user + "/friends")
    friends = dict()
    while(True):
        for friend in resource['data']:
            friends[friend['id']] = friend['name']
            # Attempt to make a request to the next page of data, if it exists.
        try:
            resource=req.get(resource['paging']['next']).json()
        except KeyError:
            #print("Finished getting friends from user " + user)
            # When there are no more pages (['paging']['next']), break from the
            # loop and end the script.
            break

    return friends

In [18]:
def getUsersDict(user, token):
    friends = getFriends(user, token)
    ffriends = dict() 
    listFFriends = []
    for idFriend in friends: 
        friendsOfFriends = getFriends(idFriend, token)
        for idUser, name in friendsOfFriends.items():
            if not idUser in ffriends and not idUser in friends:
                ffriends[idUser] = name
                listFFriends.append([idUser, name])
    
    count = 0
    while len(listFFriends) < 400 and len(listFFriends) > count:
        curruentID = listFFriends[count][0]
        friendsCurrentID = getFriends(curruentID, token)
        for idUser, name in friendsOfFriends.items():
            if not idUser in ffriends:
                ffriends[idUser] = name
                listFFriends.append([idUser, name])
        count+=1
    return ffriends

In [19]:
myfriends = getFriends('me', token)

In [20]:
#users = getUsersDict('me', token)

In [21]:
cats = getCats(token)
classf = norm(classify(cats))

otherUserPages = getPagesList(otherPersonID, token)

In [22]:
def getUserNameFromID(id, token):
    res = graph.get_object("/" + user + "?fields=name")
    return res["name"]

In [23]:
def getUserName(sortedDF, users):
    name = dict()
    for id, row in sortedDF.iterrows():
        name[id] = users[id]
    sortedDF['user-name'] = sortedDF['user-id'].map(users)
    return sortedDF
    

In [24]:
def calcSimilarityAndAddTo(key, classf, token, similarityDict):
    ratio = similarityWith(key, classf, token)
    #if(ratio > 0.000):
    similarityDict[key] = ratio
    return ratio

In [25]:
import threading
lock = threading.Lock()

def simCalcFunction(key, classf, token, similarityDict, threadsRunning):
    lock.acquire()
    threadsRunning = threadsRunning + 1
    lock.release()
    
    ratio = calcSimilarityAndAddTo(key, classf, token, similarityDict)
    
    lock.acquire()
    #print(key + " similarity with me is " + str(ratio))
    threadsRunning = threadsRunning - 1
    lock.release()

In [27]:
count = 0
limit = 32
similarityDict = dict()
threadsRunning = 0
threads = list()
for key, value in myfriends.items():
    while(threadsRunning > limit):
        i = 1
        #do nothing
    t = threading.Thread(target=simCalcFunction, args = (key, classf, token, similarityDict, threadsRunning))
    t.daemon = False
    threads.append(t)
    t.start()
    
for x in threads:
     x.join()

started thread for user 10208935375967359
started thread for user 862034200484130
started thread for user 998100796886441
started thread for user 100000888984231
started thread for user 967446623299200
started thread for user 773541116034720
started thread for user 739425496128479
started thread for user 100002211536120
started thread for user 765831436837419
started thread for user 100002541369604
started thread for user 653306944773428
started thread for user 607827396001201
started thread for user 644347655699194


In [39]:
import pandas as pd

s = pd.Series(similarityDict, name='similarity')
s.index.name = 'user-id'
s.reset_index()
df_similarity = s.to_frame()
sortedDF = df_similarity.sort_values(["similarity"], ascending=[False])

usersSeries = pd.Series(myfriends, name='user-name')
usersSeries.index.name = 'user-id'
usersSeries.reset_index()
usersDF = usersSeries.to_frame()

sortedDF['user-name'] = usersDF['user-name']
print (sortedDF)

                   similarity           user-name
user-id                                          
765831436837419          71.0        Ana Caroline
998100796886441          57.0         Ikaro Souza
739425496128479          42.0           Rai Vitor
773541116034720          23.0        Murilo Bento
862034200484130          22.0        Ivan Alisson
100002541369604          13.0  Aryan Dantas Gomes
100002211536120          12.0       Thuize Thainá
967446623299200           8.0         Paula Lopes
644347655699194           7.0       Kamilla Sâmia
653306944773428           5.0      Danilo Martins
10208935375967359         3.0     Gilberto Soares
100000888984231           0.0        Luan Fonseca
607827396001201           0.0     Inacio Medeiros
